In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

In [14]:
from html.parser import HTMLParser
import requests
from bs4 import BeautifulSoup as bs
import re
import random
import os.path

In [ ]:
insta_url='https://www.instagram.com'
inta_username= input('enter username of instagram : ')
 
response = requests.get(f"{insta_url}/{inta_username}/")

In [ ]:
# loop th

In [13]:
# # Launch a new Chrome, install the appropriate ChromeDriver if necessary
# driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
# The following should extract all images from a given page and write it to the directory where the script is being run.
# https://stackoverflow.com/questions/18408307/how-to-extract-and-download-all-images-from-a-website-using-beautifulsoup

import re
import requests
from bs4 import BeautifulSoup

site = 'http://pixabay.com'

response = requests.get(site)

soup = BeautifulSoup(response.text, 'html.parser')
img_tags = soup.find_all('img')

urls = [img['src'] for img in img_tags]


for url in urls:
    filename = re.search(r'/([\w_-]+[.](jpg|gif|png))$', url)
    if not filename:
         print("Regex didn't match with the url: {}".format(url))
         continue
    with open(filename.group(1), 'wb') as f:
        if 'http' not in url:
            # sometimes an image source can be relative 
            # if it is provide the base url which also happens 
            # to be the site variable atm. 
            url = '{}{}'.format(site, url)
        response = requests.get(url)
        f.write(response.content)

In [10]:
# Scrap-Images-from-Websites-using-BeautifulSoup
# https://github.com/nikhilroxtomar/Scrap-Images-from-Websites-using-BeautifulSoup/blob/master/save_images.py

In [ ]:
# pip install scikit-image 

In [12]:
import os
import requests
from bs4 import BeautifulSoup
import cv2
from skimage import io

In [ ]:

import os
import requests
from bs4 import BeautifulSoup
import cv2
from skimage import io

def create_dir(path):
    """ Create folders """
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print("Error")

def create_file(path):
    """ Create a file """
    try:
        if not os.path.exists(path):
            f = open(path, "w")
            f.write("Name,Alt\n")
            f.close()
    except OSError:
        print("Error")

def save_image(search_term, page_num=1):
    ## URL and headers
    url = "https://www.henrycuellar.com/"
    header = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}

    ## making a GET request to the website and getting the information in response.
    result = requests.get(url, headers=header)

    if result.status_code == 200:
        soup = BeautifulSoup(result.content, "html.parser")
    else:
        print("Error")
        exit()

    ## Paths and file for saving the images and data.
    dir_path = f"Downloads/{search_term}/"
    file_path = f"Downloads/{search_term}/{search_term}.csv"

    create_dir(dir_path)
    create_file(file_path)

    f = open(file_path, "a")

    for tag in soup.find_all("a", class_="showcase__link"):
        if tag.img:
            try:
                src = tag.img["data-src"]
                alt = tag.img["alt"]
            except Exception as e:
                alt = None

            try:
                if alt:
                    image = io.imread(src)
                    name = src.split("/")[-1].split("?")[0]
                    data = f"{name},{alt}\n"
                    f.write(data)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    cv2.imwrite(dir_path + name, image)
                    print(name, ": ", alt)
            except Exception as e:
                pass


if __name__ == "__main__":
    terms = ['dog', 'cat', 'tree']
    for term in terms:
        save_image(term, page_num=1)
        save_image(term, page_num=2)

In [9]:
import os
from subprocess import Popen, PIPE
from selenium import webdriver

In [ ]:
import os
from subprocess import Popen, PIPE
from selenium import webdriver

abspath = lambda *p: os.path.abspath(os.path.join(*p))
ROOT = abspath(os.path.dirname(__file__))


def execute_command(command):
    result = Popen(command, shell=True, stdout=PIPE).stdout.read()
    if len(result) > 0 and not result.isspace():
        raise Exception(result)


def do_screen_capturing(url, screen_path, width, height):
    print "Capturing screen.."
    driver = webdriver.PhantomJS()
    # it save service log file in same directory
    # if you want to have log file stored else where
    # initialize the webdriver.PhantomJS() as
    # driver = webdriver.PhantomJS(service_log_path='/var/log/phantomjs/ghostdriver.log')
    driver.set_script_timeout(30)
    if width and height:
        driver.set_window_size(width, height)
    driver.get(url)
    driver.save_screenshot(screen_path)


def do_crop(params):
    print "Croping captured image.."
    command = [
        'convert',
        params['screen_path'],
        '-crop', '%sx%s+0+0' % (params['width'], params['height']),
        params['crop_path']
    ]
    execute_command(' '.join(command))


def do_thumbnail(params):
    print "Generating thumbnail from croped captured image.."
    command = [
        'convert',
        params['crop_path'],
        '-filter', 'Lanczos',
        '-thumbnail', '%sx%s' % (params['width'], params['height']),
        params['thumbnail_path']
    ]
    execute_command(' '.join(command))


def get_screen_shot(**kwargs):
    url = kwargs['url']
    width = int(kwargs.get('width', 1024)) # screen width to capture
    height = int(kwargs.get('height', 768)) # screen height to capture
    filename = kwargs.get('filename', 'screen.png') # file name e.g. screen.png
    path = kwargs.get('path', ROOT) # directory path to store screen

    crop = kwargs.get('crop', False) # crop the captured screen
    crop_width = int(kwargs.get('crop_width', width)) # the width of crop screen
    crop_height = int(kwargs.get('crop_height', height)) # the height of crop screen
    crop_replace = kwargs.get('crop_replace', False) # does crop image replace original screen capture?

    thumbnail = kwargs.get('thumbnail', False) # generate thumbnail from screen, requires crop=True
    thumbnail_width = int(kwargs.get('thumbnail_width', width)) # the width of thumbnail
    thumbnail_height = int(kwargs.get('thumbnail_height', height)) # the height of thumbnail
    thumbnail_replace = kwargs.get('thumbnail_replace', False) # does thumbnail image replace crop image?

    screen_path = abspath(path, filename)
    crop_path = thumbnail_path = screen_path

    if thumbnail and not crop:
        raise Exception, 'Thumnail generation requires crop image, set crop=True'

    do_screen_capturing(url, screen_path, width, height)

    if crop:
        if not crop_replace:
            crop_path = abspath(path, 'crop_'+filename)
        params = {
            'width': crop_width, 'height': crop_height,
            'crop_path': crop_path, 'screen_path': screen_path}
        do_crop(params)

        if thumbnail:
            if not thumbnail_replace:
                thumbnail_path = abspath(path, 'thumbnail_'+filename)
            params = {
                'width': thumbnail_width, 'height': thumbnail_height,
                'thumbnail_path': thumbnail_path, 'crop_path': crop_path}
            do_thumbnail(params)
    return screen_path, crop_path, thumbnail_path


if __name__ == '__main__':
    '''
        Requirements:
        Install NodeJS
        Using Node's package manager install phantomjs: npm -g install phantomjs
        install selenium (in your virtualenv, if you are using that)
        install imageMagick
        add phantomjs to system path (on windows)
    '''

    url = 'http://stackoverflow.com/questions/1197172/how-can-i-take-a-screenshot-image-of-a-website-using-python'
    screen_path, crop_path, thumbnail_path = get_screen_shot(
        url=url, filename='sof.png',
        crop=True, crop_replace=False,
        thumbnail=True, thumbnail_replace=False,
        thumbnail_width=200, thumbnail_height=150,
    )

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

site = 'http://pixabay.com'

response = requests.get(site)

soup = BeautifulSoup(response.text, 'html.parser')
img_tags = soup.find_all('img')

urls = [img['src'] for img in img_tags]


for url in urls:
    filename = re.search(r'/([\w_-]+[.](jpg|gif|png))$', url)
    if not filename:
         print("Regex didn't match with the url: {}".format(url))
         continue
    with open(filename.group(1), 'wb') as f:
        if 'http' not in url:
            # sometimes an image source can be relative 
            # if it is provide the base url which also happens 
            # to be the site variable atm. 
            url = '{}{}'.format(site, url)
        response = requests.get(url)
        f.write(response.content)

In [6]:
import os
  
list = ['folder10','folder11','folder12',
        'folder13', 'folder15']
  
for items in list:
    os.mkdir(items)

In [5]:
# df = pd.read_csv("outputs//output_house_candidate_campaign_links.csv", encoding="utf-8")
# df

In [ ]:
# how to extract all images on a website

# https://stackoverflow.com/questions/18408307/how-to-extract-and-download-all-images-from-a-website-using-beautifulsoup

In [ ]:


# site = 'http://www.google.com'

# response = requests.get(site)

# soup = BeautifulSoup(response.text, 'html.parser')
# img_tags = soup.find_all('img')

# urls = [img['src'] for img in img_tags]


# for url in urls:
#     filename = re.search(r'/([\w_-]+[.](jpg|gif|png))$', url)
#     if not filename:
#          print("Regex didn't match with the url: {}".format(url))
#          continue
#     with open(filename.group(1), 'wb') as f:
#         if 'http' not in url:
#             # sometimes an image source can be relative 
#             # if it is provide the base url which also happens 
#             # to be the site variable atm. 
#             url = '{}{}'.format(site, url)
#         response = requests.get(url)
#         f.write(response.content)

In [ ]:
# https://betterprogramming.pub/how-to-easily-scrape-multiple-pages-of-a-website-using-python-73e85bd06f8c

In [ ]:
# Exporting data in a txt file

# with open(f'{title}.txt', 'w') as file:
#     file.write(transcript)

In [ ]:
# https://github.com/adityamangal1/Image-Scraping-Python/blob/master/main.py

In [ ]:
# import bs4
# import requests
# import shutil
# import os

# GOOGLE_IMAGE = \
#     'https://www.google.com/search?site=&tbm=isch&source=hp&biw=1873&bih=990&'


# def extract(data, quantity):
#     URL_input = GOOGLE_IMAGE + 'q=' + data
#     print('Fetching from url =', URL_input)
#     URLdata = requests.get(URL_input)
#     soup = bs4.BeautifulSoup(URLdata.text, "html.parser")
#     ImgTags = soup.find_all('img')
#     i = 0
#     print('Please wait..')
#     while i < quantity:

#         for link in ImgTags:
#             try:
#                 images = link.get('src')
#                 ext = images[images.rindex('.'):]
#                 if ext.startswith('.png'):
#                     ext = '.png'
#                 elif ext.startswith('.jpg'):
#                     ext = '.jpg'
#                 elif ext.startswith('.jfif'):
#                     ext = '.jfif'
#                 elif ext.startswith('.com'):
#                     ext = '.jpg'
#                 elif ext.startswith('.svg'):
#                     ext = '.svg'
#                 data = requests.get(images, stream=True)
#                 filename = str(i) + ext
#                 with open(filename, 'wb') as file:
#                     shutil.copyfileobj(data.raw, file)
#                 i += 1
#             except:
#                 pass
#     print('\t\t\t Downloaded Successfully..\t\t ')


# data = input('What are you looking for? ')
# quantity = int(input('How many photos you want? '))
# extract(data, quantity)

In [ ]:
# https://github.com/nikhilroxtomar/Scrap-Images-from-Websites-using-BeautifulSoup/blob/master/save_images.py
